In [ ]:
import glob
import re
import pandas as pd

In [ ]:
##
## make dataframe of working areas
##

In [ ]:
work_areas = glob.glob('work_areas/crab_201*')

In [ ]:
def is_data(string):
    return 'data' in string
def get_era(string):
    eras = re.findall(r'(201[0-9])', string)
    if len(eras)==1: return eras[0]
    return 0

In [ ]:
working_areas  = [{'area':w,'data': is_data(w), 'era':get_era(w)} for w in work_areas]
working_areas_df = pd.DataFrame(working_areas)

In [ ]:
##
## test crab status
##

In [ ]:
import os

In [ ]:
def crab_read(string):
    stream = os.popen('crab status -d {}'.format(string))
    return stream.read()
test_string = 'work_areas/crab_2016_WJets_WJetsToQQ_HT_WJetsToQQ_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphM_deepflavour_bff'

In [ ]:
def get_status(string):
    results = re.findall(r'Jobs status: [ ]+([a-zA-Z]+)', string)
    if len(results)==1: return results[0]
    return 0
def job_counts(string):
    result = re.findall(r'\(([0-9]+)\/([0-9]+)\)', string)
    if len(result)==1: return result[0]
    return (0,0)
def jobs_finished(string):
    return job_counts(string)[0]
def jobs_total(string):
    return job_counts(string)[1]

In [ ]:
def make_results(row):
    return crab_read(row.area)

In [ ]:
#get results for each row
result_collumn = working_areas_df.apply(make_results, axis=1)

In [ ]:
working_areas_df['results'] = result_collumn

In [ ]:
working_areas_df['status'] = result_collumn.apply(get_status)
working_areas_df['jobs_finished'] = result_collumn.apply(jobs_finished)
working_areas_df['jobs_total'] = result_collumn.apply(jobs_total)

In [ ]:
#print completed
count = 0
for i,row in working_areas_df.iterrows():
    if row.status=='finished' and row.jobs_finished==row.jobs_total:
        count += 1
        print(count, row.area)

In [ ]:
#print not complted
count = 0
for i,row in working_areas_df.iterrows():
    if not (row.status=='finished' and row.jobs_finished==row.jobs_total):
        count += 1
        print count, "\t", row.area, "\t",  row.status, "\t",  row.jobs_finished, "\t",  row.jobs_total 

In [ ]:
#print failed
count = 0
for i,row in working_areas_df.iterrows():
    if (row.status=='failed'):
        count += 1
        print(count, row.area)

In [ ]:
working_areas_df.to_csv('skim_status.csv')